# QA Comparison: BERT-style Extractive vs RAG-style Generative

In [ ]:
# !pip install transformers torch scikit-learn

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np

c:\Proj\RAG\rag-demo-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Proj\RAG\rag-demo-env\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Proj\RAG\rag-demo-env\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


## Example text and question

In [59]:
context = '''ProductX is the latest widget released in 2024. It features improved battery life.
To reset ProductX, hold the power button for 10 seconds until the LED blinks.
Our support plans include Basic, Plus, and Enterprise tiers. ''' 
# context = [
#     "ProductX is the latest widget released in 2024. It features improved battery life.",
#     "To reset ProductX, hold the power button for 10 seconds until the LED blinks.",
#     "Our support plans include Basic, Plus, and Enterprise tiers, offering 24/7 support in higher tiers."
# ]
question = 'How many plans are there   ?'

## 🔶 Part 1: BERT-style extractive QA

In [60]:
bert_qa = pipeline('question-answering', model='distilbert-base-cased-distilled-squad') 
bert_result = bert_qa(question=question, context=context) 
print(f"Extractive Answer: {bert_result['answer']}")

Extractive Answer: Basic, Plus, and Enterprise tiers


## 🟢 Part 2: RAG-style semantic retrieval + generative QA

In [61]:
# Embedding model\n
embed_model_name = 'sentence-transformers/all-MiniLM-L6-v2' 
embed_tokenizer = AutoTokenizer.from_pretrained(embed_model_name) 
embed_model = AutoModel.from_pretrained(embed_model_name)

In [62]:
def mean_pooling(model_output, attention_mask): 
    token_embeddings = model_output[0]  
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float() 
    return (token_embeddings * input_mask_expanded).sum(1) / input_mask_expanded.sum(1)

In [63]:
def embed(text): 
    tokens = embed_tokenizer([text], return_tensors='pt', padding=True, truncation=True) 
    with torch.no_grad(): 
        output = embed_model(**tokens) 
    return mean_pooling(output, tokens['attention_mask']).numpy()

In [64]:
# Split context into sentences\n
sentences = context.strip().split(".") 
sentence_embeddings = np.vstack([embed(s) for s in sentences]) 
query_embedding = embed(question)[0]
print("query shape:", query_embedding.shape)
print("sentences shape:", sentence_embeddings.shape)

similarities = cosine_similarity([query_embedding], sentence_embeddings)[0]  
best_idx = int(np.argmax(similarities)) 
retrieved = sentences[best_idx].strip() 
print(f"Retrieved for RAG: {retrieved}")

query shape: (384,)
sentences shape: (5, 384)
Retrieved for RAG: Our support plans include Basic, Plus, and Enterprise tiers


In [65]:
query_embedding.shape
sentence_embeddings.shape

(5, 384)

In [66]:
# Generation model\n
gen_pipeline = pipeline('text2text-generation', model='google/flan-t5-base') 
prompt = f"Context: {retrieved} Question: {question} Answer:" 
gen_result = gen_pipeline(prompt, max_length=100)[0]['generated_text'] 
print(f"Generative Answer: {gen_result}")

c:\Proj\RAG\rag-demo-env\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generative Answer: three
